In [1]:
#Import Python Packages
#from google.colab import drive
#drive.mount('/content/drive/')

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
import pandas as pd
import numpy as np
from scipy.stats import randint
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report

#load the training dataset

trainfile =r'/gdrive/My Drive/CIS_508/Datasets/Insurance Fraud - TRAIN-3000(1).csv'
train=pd.read_csv(trainfile)
train.head()

#load the test data set

testfile=r'/gdrive/My Drive/CIS_508/Datasets/Insurance Fraud -TEST-12900(1).csv'
test=pd.read_csv(testfile)
test.head()

#copy train and test data

traincopy=train.iloc[:,:-1].copy()
testcopy=test.iloc[:,:-1].copy()

traincopy.head()
testcopy.head()

#lets find the categorical variables

categorical_features=traincopy.columns[traincopy.dtypes==object].tolist()
print(categorical_features)

#combination of train and test data

combination=pd.concat([traincopy,testcopy],keys=[0,1])

onehotencoded=pd.get_dummies(combination,columns=categorical_features)

x_train=onehotencoded.xs(0)
x_test=onehotencoded.xs(1)
 
print(x_train.shape)
print(x_test.shape)

['MONTH', 'DAYOFWEEK', 'MAKE', 'ACCIDENTAREA', 'DAYOFWEEKCLAIMED', 'MONTHCLAIMED', 'SEX', 'MARITALSTATUS', 'FAULT', 'POLICYTYPE', 'VEHICLECATEGORY', 'VEHICLEPRICE', 'DAYS_POLICY_ACCIDENT', 'DAYS_POLICY_CLAIM', 'PASTNUMBEROFCLAIMS', 'AGEOFVEHICLE', 'AGEOFPOLICYHOLDER', 'POLICEREPORTFILED', 'WITNESSPRESENT', 'AGENTTYPE', 'NUMBEROFSUPPLIMENTS', 'ADDRESSCHANGE_CLAIM', 'NUMBEROFCARS', 'BASEPOLICY']
(2999, 144)
(12918, 144)


In [0]:
#define the parameters

param_grid={"max_depth": range(1,10,3),
                    "max_leaf_nodes":range(2,20,5),
                                "min_samples_leaf" :range(2,10,2),
                                  "max_features":range(1,30,5),
                                    "criterion":('gini','entropy'),
}

In [0]:
y_train=train.iloc[:,-1]
y_test=test.iloc[:,-1]

In [5]:
#select the target(fraudfound) - GRIDSEARCH



rclass=DecisionTreeClassifier()

grid_tree=GridSearchCV(rclass,param_grid,cv=3)

grid_tree.fit(x_train,y_train)

print("After hypertunig parameters with GridsearchCV:{}".format(grid_tree.best_params_))
print("Highest accuracy score we get is:{}".format(grid_tree.best_score_))
print("Best estimator are:{}".format(grid_tree.best_estimator_))
print("Area under ROC curve:{}".format(grid_tree.best_index_))
print("The number of cross-validation splits:{}".format(grid_tree.n_splits_))


After hypertunig parameters with GridsearchCV:{'criterion': 'entropy', 'max_depth': 4, 'max_features': 26, 'max_leaf_nodes': 12, 'min_samples_leaf': 6}
Highest accuracy score we get is:0.8866288762920974
Best estimator are:DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=26, max_leaf_nodes=12,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
Area under ROC curve:474
The number of cross-validation splits:3


In [0]:
rclass.get_params().keys()

dict_keys(['class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'presort', 'random_state', 'splitter'])

In [0]:

prediction=pd.DataFrame(grid_tree.predict(x_train),columns=["FRAUDFOUND"])
#Print Classification Report
print(classification_report(y_train,prediction))

              precision    recall  f1-score   support

          No       0.88      0.99      0.93      2600
         Yes       0.62      0.11      0.19       399

    accuracy                           0.87      2999
   macro avg       0.75      0.55      0.56      2999
weighted avg       0.84      0.87      0.83      2999



In [0]:
prediction_test=pd.DataFrame(grid_tree.predict(x_test),columns=['FRAUDFOUND'])
print(classification_report(y_test,prediction_test))

              precision    recall  f1-score   support

          No       0.96      0.98      0.97     12420
         Yes       0.13      0.09      0.11       498

    accuracy                           0.94     12918
   macro avg       0.55      0.53      0.54     12918
weighted avg       0.93      0.94      0.94     12918



In [0]:
print("Confusion matrix is as follows:")
print(confusion_matrix(y_test,prediction_test))

Confusion matrix is as follows:
[[10793  1627]
 [  305   193]]


In [0]:
#model - 2 - trying differnet ranges for grid search

#define the parameters

param_grid={
                    "max_leaf_nodes":range(2,200,50),
                              "min_samples_leaf" :range(2,100,20),
                             
                                    "criterion":('gini','entropy'),
}


#select the target(fraudfound) - GRIDSEARCH



rclass=DecisionTreeClassifier()

grid_tree=GridSearchCV(rclass,param_grid,cv=3)

grid_tree.fit(x_train,y_train)

print("After hypertunig parameters with GridsearchCV:{}".format(grid_tree.best_params_))
print("Highest accuracy score we get is:{}".format(grid_tree.best_score_))
print("Best estimator are:{}".format(grid_tree.best_estimator_))
print("Area under ROC curve:{}".format(grid_tree.best_index_))
print("The number of cross-validation splits:{}".format(grid_tree.n_splits_))

prediction=pd.DataFrame(grid_tree.predict(x_train),columns=["FRAUDFOUND"])
#Print Classification Report
print(classification_report(y_train,prediction))

prediction_test=pd.DataFrame(grid_tree.predict(x_test),columns=['FRAUDFOUND'])
print(classification_report(y_test,prediction_test))


print("Confusion matrix is as follows:")
print(confusion_matrix(y_test,prediction_test))





After hypertunig parameters with GridsearchCV:{'criterion': 'gini', 'max_features': 3, 'max_leaf_nodes': 152, 'min_samples_leaf': 22}
Highest accuracy score we get is:0.8729576525508503
Best estimator are:DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=3, max_leaf_nodes=152,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=22, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
Area under ROC curve:56
The number of cross-validation splits:3
              precision    recall  f1-score   support

          No       0.87      0.99      0.93      2600
         Yes       0.52      0.04      0.07       399

    accuracy                           0.87      2999
   macro avg       0.69      0.52      0.50      2999
weighted avg       0.82      0.87      0.81      2999

  

In [0]:
#model - 3 - trying differnet ranges for grid search

#define the parameters

param_grid={ 
                    "max_leaf_nodes":range(2,100,40),
                              "min_samples_leaf" :range(2,80,40),
                                 "max_features":range(1,5,3),
                                    "criterion":('gini','entropy'),
}


#select the target(fraudfound) - GRIDSEARCH



rclass=DecisionTreeClassifier()

grid_tree=GridSearchCV(rclass,param_grid,cv=3)

grid_tree.fit(x_train,y_train)

print("After hypertunig parameters with GridsearchCV:{}".format(grid_tree.best_params_))
print("Highest accuracy score we get is:{}".format(grid_tree.best_score_))
print("Best estimator are:{}".format(grid_tree.best_estimator_))
print("Area under ROC curve:{}".format(grid_tree.best_index_))
print("The number of cross-validation splits:{}".format(grid_tree.n_splits_))

prediction=pd.DataFrame(grid_tree.predict(x_train),columns=["FRAUDFOUND"])
#Print Classification Report
print(classification_report(y_train,prediction))

prediction_test=pd.DataFrame(grid_tree.predict(x_test),columns=['FRAUDFOUND'])
print(classification_report(y_test,prediction_test))


print("Confusion matrix is as follows:")
print(confusion_matrix(y_test,prediction_test))



After hypertunig parameters with GridsearchCV:{'criterion': 'entropy', 'max_features': 1, 'max_leaf_nodes': 82, 'min_samples_leaf': 2}
Highest accuracy score we get is:0.8679559853284429
Best estimator are:DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=1, max_leaf_nodes=82,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
Area under ROC curve:16
The number of cross-validation splits:3
              precision    recall  f1-score   support

          No       0.87      1.00      0.93      2600
         Yes       0.67      0.03      0.05       399

    accuracy                           0.87      2999
   macro avg       0.77      0.51      0.49      2999
weighted avg       0.84      0.87      0.81      2999



In [0]:
#model - 4 - with scoring as precision

#define the parameters

param_grid={ 
                    "max_leaf_nodes":range(2,100,40),
                              "min_samples_leaf" :range(2,80,40),
                                 "max_features":range(1,5,3),
                                    "criterion":('gini','entropy'),
}


#select the target(fraudfound) - GRIDSEARCH



rclass=DecisionTreeClassifier()

grid_tree=GridSearchCV(rclass,param_grid,cv=3,scoring='accuracy')

grid_tree.fit(x_train,y_train)

print("After hypertunig parameters with GridsearchCV:{}".format(grid_tree.best_params_))
print("Highest accuracy score we get is:{}".format(grid_tree.best_score_))
print("Best estimator are:{}".format(grid_tree.best_estimator_))
print("Area under ROC curve:{}".format(grid_tree.best_index_))
print("The number of cross-validation splits:{}".format(grid_tree.n_splits_))

prediction=pd.DataFrame(grid_tree.predict(x_train),columns=["FRAUDFOUND"])
#Print Classification Report
print(classification_report(y_train,prediction))

prediction_test=pd.DataFrame(grid_tree.predict(x_test),columns=['FRAUDFOUND'])
print(classification_report(y_test,prediction_test))


print("Confusion matrix is as follows:")
print(confusion_matrix(y_test,prediction_test))


After hypertunig parameters with GridsearchCV:{'criterion': 'entropy', 'max_features': 4, 'max_leaf_nodes': 42, 'min_samples_leaf': 2}
Highest accuracy score we get is:0.8672890963654551
Best estimator are:DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=4, max_leaf_nodes=42,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
Area under ROC curve:20
The number of cross-validation splits:3
              precision    recall  f1-score   support

          No       0.88      0.99      0.94      2600
         Yes       0.76      0.15      0.25       399

    accuracy                           0.88      2999
   macro avg       0.82      0.57      0.59      2999
weighted avg       0.87      0.88      0.84      2999



In [0]:
#implementation of RandomForest

ranf = RandomForestClassifier(n_estimators=90, 
                              n_jobs=-1,
                               bootstrap = True,
                               max_features = 'sqrt',verbose=1)

ranf.fit(x_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    0.3s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=90, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=1,
                       warm_start=False)

In [0]:
#Average number of nodes  & Average maximum depth
n_nodes = []
max_depths = []

for ind_tree in ranf.estimators_:
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

Average number of nodes 807
Average maximum depth 26


In [0]:
prediction_ranf=pd.DataFrame(ranf.predict(x_train),columns=["FRAUDFOUND"])
#Print Classification Report
print(classification_report(y_train,prediction_ranf))

              precision    recall  f1-score   support

          No       1.00      1.00      1.00      2600
         Yes       1.00      1.00      1.00       399

    accuracy                           1.00      2999
   macro avg       1.00      1.00      1.00      2999
weighted avg       1.00      1.00      1.00      2999



[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  90 out of  90 | elapsed:    0.1s finished


In [0]:
prediction_ranf_test=pd.DataFrame(ranf.predict(x_test),columns=['FRAUDFOUND'])
print(classification_report(y_test,prediction_ranf_test))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  80 out of  80 | elapsed:    0.1s finished


              precision    recall  f1-score   support

          No       0.99      0.97      0.98     12420
         Yes       0.51      0.85      0.63       498

    accuracy                           0.96     12918
   macro avg       0.75      0.91      0.81     12918
weighted avg       0.97      0.96      0.97     12918



In [0]:
print(confusion_matrix(y_test,prediction_ranf_test))


[[11947   473]
 [   78   420]]


In [0]:
#implementation of random forest with hyper parameter tunig
param_random_rf={'n_estimators': np.linspace(10, 200).astype(int),
    'max_depth': [None] + list(np.linspace(3, 30).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True, False]}

In [0]:
ranf_hyper=RandomForestClassifier()

In [0]:
from sklearn.model_selection import RandomizedSearchCV

rs_rf= RandomizedSearchCV(ranf_hyper,param_random_rf,cv=3,n_iter=15,verbose=1)

rs_rf.fit(x_train,y_train)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   29.8s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [0]:
prediction_ranf_cv=pd.DataFrame(rs_rf.predict(x_train),columns=["FRAUDFOUND"])
#Print Classification Report
print(classification_report(y_train,prediction_ranf_cv))

              precision    recall  f1-score   support

          No       0.90      1.00      0.95      2600
         Yes       0.98      0.27      0.42       399

    accuracy                           0.90      2999
   macro avg       0.94      0.63      0.68      2999
weighted avg       0.91      0.90      0.88      2999



In [0]:
prediction_ranf_cv=pd.DataFrame(rs_rf.predict(x_test),columns=["FRAUDFOUND"])
#Print Classification Report
print(classification_report(y_test,prediction_ranf_cv))

              precision    recall  f1-score   support

          No       0.97      0.98      0.97     12420
         Yes       0.30      0.25      0.27       498

    accuracy                           0.95     12918
   macro avg       0.63      0.61      0.62     12918
weighted avg       0.94      0.95      0.95     12918



In [0]:
print(confusion_matrix(y_test,prediction_ranf_cv))

[[12120   300]
 [  372   126]]


In [0]:
#implementation of random forest with hyper parameter tunig grid search
param_grid_rf=param_grid_rf={'n_estimators': np.linspace(1, 2).astype(int),
    'max_depth': [None] + list(np.linspace(3, 4).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1))}

In [0]:
ranf_hyper_grid=RandomForestClassifier()
from sklearn.model_selection import GridSearchCV
rf_grid= GridSearchCV(ranf_hyper_grid,param_grid_rf,cv=3,scoring="balanced_accuracy")

rf_grid.fit(x_train,y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=Fa...
             param_grid={'max_depth': [None, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
                                       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [0]:
prediction_ranf_grid=pd.DataFrame(rf_grid.predict(x_train),columns=["FRAUDFOUND"])
#Print Classification Report
print(classification_report(y_train,prediction_ranf_grid))

              precision    recall  f1-score   support

          No       0.90      0.98      0.93      2600
         Yes       0.64      0.26      0.37       399

    accuracy                           0.88      2999
   macro avg       0.77      0.62      0.65      2999
weighted avg       0.86      0.88      0.86      2999



In [0]:
prediction_ranf_grid=pd.DataFrame(rf_grid.predict(x_test),columns=["FRAUDFOUND"])
print(classification_report(y_test,prediction_ranf_grid))
print(confusion_matrix(y_test,prediction_ranf_grid))

              precision    recall  f1-score   support

          No       0.97      0.91      0.94     12420
         Yes       0.10      0.25      0.15       498

    accuracy                           0.89     12918
   macro avg       0.54      0.58      0.54     12918
weighted avg       0.93      0.89      0.91     12918

[[11345  1075]
 [  372   126]]
